# **WEEK 5**

The problem is to find the best places (activities, restaurants..) for a given city, for example Paris. We will list and vizualize them.



### Import libraries

In [26]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
%matplotlib inline 
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# Libraries
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes 

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup
print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


### Data traitement

In [27]:
paris_df = pd.read_csv("https://lvdesign.com.fr/coursera_data/paris.csv")
paris_df.rename(columns= {'Lieux':'Places'},inplace= True)
paris_df.head()

,Places,Latitude,Longitude
0,Musée du Louvre,48.860611,2.335450
1,Notre Dame de Paris,48.852968,2.347708
2,Arc de Triomphe,48.873792,2.295028
3,Tour Eiffel,48.858370,2.294481


### Paris location and exploration

In [28]:
address = 'Paris'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [29]:
paris_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(paris_df['Latitude'], paris_df['Longitude'], paris_df['Places']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_map)  
    
paris_map

### Foursquare Data

In [30]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'RCXUHDTW1HEDQVR0I1PGX4MKW242T3CA34OFKP2FPPXBD4OT' # your Foursquare ID
CLIENT_SECRET = 'ZT2DUOHWPMZOJ02S4F4CE5AETRGXK1I52QRUXQDQJFKQSMV4' # your Foursquare Secret
VERSION = '20180602' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RCXUHDTW1HEDQVR0I1PGX4MKW242T3CA34OFKP2FPPXBD4OT
CLIENT_SECRET:ZT2DUOHWPMZOJ02S4F4CE5AETRGXK1I52QRUXQDQJFKQSMV4


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,            
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Lieux', 
                  'Lieux Latitude', 
                  'Lieux Longitude', 
                  'Venue',
                  'Venue id',                
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'
                             
                   ]
    
    return(nearby_venues)

In [32]:

venue_id ={
    'Louvres Museum': '4adcda10f964a520af3521e3',
    'Notre-Dame de Paris cathedral': '4adcda09f964a520e83321e3',
    'Eiffel Tower': '51a2445e5019c80b56934c75',
    'Arc de Triomphe': '4adcda09f964a520de3321e3'
}

for key,value in venue_id.items():
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(value, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print('Venue :', key)
        print('Rating : ', result['response']['venue']['rating'])
        print('Total Tips : ', result['response']['venue']['tips']['count'])
        print('Nb Likes : ', result['response']['venue']['tips']['groups'][0]['items'][0]['likes']['count'])
        print('Nb Likes / Total Tips', (result['response']['venue']['tips']['groups'][0]['items'][0]['likes']['count'])/ (result['response']['venue']['tips']['count']) *100 )
        
        print()

    except:
        print('This venue has not been rated yet.')


Venue : Louvres Museum
Rating :  9.3
Total Tips :  2270
Nb Likes :  167
Nb Likes / Total Tips 7.356828193832598

Venue : Notre-Dame de Paris cathedral
Rating :  9.3
Total Tips :  981
Nb Likes :  57
Nb Likes / Total Tips 5.81039755351682

Venue : Eiffel Tower
Rating :  9.4
Total Tips :  2653
Nb Likes :  933
Nb Likes / Total Tips 35.167734640030154

Venue : Arc de Triomphe
Rating :  9.4
Total Tips :  775
Nb Likes :  102
Nb Likes / Total Tips 13.161290322580646



In [33]:
paris_data_venues = getNearbyVenues(names=paris_df['Places'],
                                   latitudes=paris_df['Latitude'],
                                   longitudes=paris_df['Longitude']
                                  )

paris_data_venues.groupby('Lieux').count()

Musée du Louvre
Notre Dame de Paris
Arc de Triomphe
Tour Eiffel


,Lieux Latitude,Lieux Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
Lieux,,,,,,,
Arc de Triomphe,69,69,69,69,69,69,69
Musée du Louvre,53,53,53,53,53,53,53
Notre Dame de Paris,62,62,62,62,62,62,62
Tour Eiffel,49,49,49,49,49,49,49


In [34]:
df_view_new = paris_data_venues.groupby('Lieux').count()
df_paris_hist = df_view_new['Venue Category']
df_paris_hist

Lieux
Arc de Triomphe        69
Musée du Louvre        53
Notre Dame de Paris    62
Tour Eiffel            49
Name: Venue Category, dtype: int64

In [37]:
paris_onehot = pd.get_dummies(paris_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add Lieux column back to dataframe
paris_onehot['Lieux'] = paris_data_venues['Lieux'] 

# move lieux column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]
paris_group = paris_onehot.groupby('Lieux').mean().reset_index()
paris_group

,Lieux,Art Museum,Arts & Crafts Store,Asian Restaurant,Bagel Shop,Bakery,Bar,Basketball Court,Beach Bar,Bistro,Bookstore,Boutique,Brasserie,Breton Restaurant,Bridge,Burger Joint,Bus Stop,Café,Candy Store,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Dessert Shop,Electronics Store,Exhibit,Falafel Restaurant,Flower Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gift Shop,Historic Site,History Museum,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Lebanese Restaurant,Library,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Nightclub,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Pizza Place,Playground,Plaza,Pub,Restaurant,Romanian Restaurant,Roof Deck,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Spa,Sports Bar,Steakhouse,Supermarket,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Track,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Wine Shop
0,Arc de Triomphe,0.000000,0.000000,0.014493,0.014493,0.028986,0.000000,0.000000,0.000000,0.014493,0.000000,0.014493,0.000000,0.014493,0.000000,0.000000,0.000000,0.014493,0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.028986,0.014493,0.000000,0.014493,0.000000,0.014493,0.014493,0.014493,0.014493,0.000000,0.028986,0.000000,0.000000,0.000000,0.000000,0.144928,0.000000,0.000000,0.000000,0.000000,0.000000,0.202899,0.000000,0.000000,0.086957,0.028986,0.014493,0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.014493,0.028986,0.000000,0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.000000,0.000000,0.000000,0.028986,0.000000,0.028986,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493,0.000000,0.000000,0.000000,0.014493,0.000000
1,Musée du Louvre,0.056604,0.018868,0.000000,0.000000,0.000000,0.018868,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000,0.000000,0.018868,0.000000,0.000000,0.000000,0.018868,0.000000,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000,0.018868,0.000000,0.000000,0.000000,0.037736,0.000000,0.000000,0.018868,0.000000,0.018868,0.000000,0.000000,0.000000,0.150943,0.018868,0.018868,0.000000,0.056604,0.000000,0.113208,0.000000,0.000000,0.056604,0.018868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037736,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000,0.000000,0.094340,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0.018868,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0.000000
2,Notre Dame de Paris,0.000000,0.016129,0.000000,0.000000,0.048387,0.048387,0.000000,0.000000,0.048387,0.048387,0.000000,0.016129,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.016129,0.016129,0.000000,0.016129,0.032258,0.000000,0.032258,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.016129,0.016129,0.016129,0.048387,0.000000,0.016129,0.000000,0.000000,0.016129,0.032258,0.016129,0.048387,0.000000,0.048387,0.000000,0.000000,0.032258,0.000000,0.016129,0.016129,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.016129,0.016129,0.000000,0.000000,0.000000,0.064516,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129
3,Tour Eiffel,0.020408,0.000000,0.000000,0.000000,0.040816,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.0408

In [41]:
num_top_venues = 15

for hood in paris_group['Lieux']:
    print("----"+hood+"----")
    temp = paris_group[paris_group['Lieux'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arc de Triomphe----
                  venue  freq
0                 Hotel  0.20
1     French Restaurant  0.14
2    Italian Restaurant  0.09
3         Movie Theater  0.03
4                   Spa  0.03
5            Steakhouse  0.03
6     Electronics Store  0.03
7        Clothing Store  0.03
8   Japanese Restaurant  0.03
9                Bakery  0.03
10        Jewelry Store  0.01
11                Plaza  0.01
12           Kids Store  0.01
13         Cocktail Bar  0.01
14       Sandwich Place  0.01


----Musée du Louvre----
                     venue  freq
0        French Restaurant  0.15
1                    Hotel  0.11
2                    Plaza  0.09
3       Italian Restaurant  0.06
4               Art Museum  0.06
5            Historic Site  0.06
6                   Museum  0.04
7           Cosmetics Shop  0.04
8   Furniture / Home Store  0.02
9                   Garden  0.02
10        Pedestrian Plaza  0.02
11            Perfume Shop  0.02
12             Coffee Shop  0.02
13      

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Lieux']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


lieux_venues_sort1= pd.DataFrame(columns=columns) # create a new dataframe
lieux_venues_sort1['Lieux'] = paris_group['Lieux']

for ind in np.arange(paris_group.shape[0]):
    lieux_venues_sort1.iloc[ind, 1:] = return_most_common_venues(paris_group.iloc[ind, :], num_top_venues)

lieux_venues_sort1

,Lieux,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Arc de Triomphe,Hotel,French Restaurant,Italian Restaurant,Japanese Restaurant,Spa,Movie Theater,Clothing Store,Bakery,Electronics Store,Steakhouse,Vegetarian / Vegan Restaurant,Plaza,Café,Sandwich Place,Candy Store
1,Musée du Louvre,French Restaurant,Hotel,Plaza,Art Museum,Historic Site,Italian Restaurant,Museum,Cosmetics Shop,Dessert Shop,Bridge,Coffee Shop,Clothing Store,Japanese Restaurant,Church,Pedestrian Plaza
2,Notre Dame de Paris,Plaza,Indie Movie Theater,Bakery,Bar,Bistro,Bookstore,French Restaurant,Japanese Restaurant,Coffee Shop,Hotel,Park,Comic Shop,Lebanese Restaurant,Burger Joint,Ice Cream Shop
3,Tour Eiffel,French Restaurant,Italian Restaurant,Hotel,Bakery,Garden,Plaza,Café,Art Museum,Romanian Restaurant,Monument / Landmark,Convenience Store,Chinese Restaurant,Theme Park Ride / Attraction,Champagne Bar,Pedestrian Plaza


### Clusters

In [53]:
clusternumber=4
paris_clustering = paris_group.drop('Lieux', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusternumber, random_state=0).fit(paris_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

paris_merged = paris_df

# add clustering labels
paris_merged['Cluster Labels'] = kmeans.labels_

paris_merged = paris_merged.join(lieux_venues_sort1.set_index('Lieux'), on='Places')
paris_merged 

,Places,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Musée du Louvre,48.860611,2.335450,0,French Restaurant,Hotel,Plaza,Art Museum,Historic Site,Italian Restaurant,Museum,Cosmetics Shop,Dessert Shop,Bridge,Coffee Shop,Clothing Store,Japanese Restaurant,Church,Pedestrian Plaza
1,Notre Dame de Paris,48.852968,2.347708,3,Plaza,Indie Movie Theater,Bakery,Bar,Bistro,Bookstore,French Restaurant,Japanese Restaurant,Coffee Shop,Hotel,Park,Comic Shop,Lebanese Restaurant,Burger Joint,Ice Cream Shop
2,Arc de Triomphe,48.873792,2.295028,1,Hotel,French Restaurant,Italian Restaurant,Japanese Restaurant,Spa,Movie Theater,Clothing Store,Bakery,Electronics Store,Steakhouse,Vegetarian / Vegan Restaurant,Plaza,Café,Sandwich Place,Candy Store
3,Tour Eiffel,48.858370,2.294481,2,French Restaurant,Italian Restaurant,Hotel,Bakery,Garden,Plaza,Café,Art Museum,Romanian Restaurant,Monument / Landmark,Convenience Store,Chinese Restaurant,Theme Park Ride / Attraction,Champagne Bar,Pedestrian Plaza


In [60]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)# create map

# set color scheme for the clusters
x = np.arange(clusternumber)
ys = [i+x+(i*x)**3 for i in range(clusternumber)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Places'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color='black',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
#LOUVRES
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,48.860611,Hotel,Plaza,Art Museum,Historic Site,Italian Restaurant,Museum,Cosmetics Shop,Dessert Shop,Bridge,Coffee Shop,Clothing Store,Japanese Restaurant,Church,Pedestrian Plaza


In [64]:
#Arc de triumphe
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,48.873792,French Restaurant,Italian Restaurant,Japanese Restaurant,Spa,Movie Theater,Clothing Store,Bakery,Electronics Store,Steakhouse,Vegetarian / Vegan Restaurant,Plaza,Café,Sandwich Place,Candy Store


In [65]:
#eiffel tower
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,48.85837,Italian Restaurant,Hotel,Bakery,Garden,Plaza,Café,Art Museum,Romanian Restaurant,Monument / Landmark,Convenience Store,Chinese Restaurant,Theme Park Ride / Attraction,Champagne Bar,Pedestrian Plaza


In [66]:
#notre dame 
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]


,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,48.852968,Indie Movie Theater,Bakery,Bar,Bistro,Bookstore,French Restaurant,Japanese Restaurant,Coffee Shop,Hotel,Park,Comic Shop,Lebanese Restaurant,Burger Joint,Ice Cream Shop


### study arc de triumphe

In [69]:
arc_latitude = paris_df.loc[2, 'Latitude'] 
arc_longitude = paris_df.loc[2, 'Longitude'] 

lieux_name = paris_df.loc[2, 'Places'] # Lieux name

print('Latitude and longitude values of {} are {}, {}.'.format(lieux_name, 
                                                               arc_latitude, 
                                                               arc_longitude))

Latitude and longitude values of Arc de Triomphe are 48.8737917, 2.2950275.


In [72]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,arc_latitude,arc_longitude,radius,LIMIT)
url # display URL

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb866e7b57e88001c4df233'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b291899f964a5209f9824e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d164941735',
         'name': 'Plaza',
         'pluralName': 'Plazas',
         'primary': True,
         'shortName': 'Plaza'}],
       'id': '4b291899f964a5209f9824e3',
       'location': {'address': 'Place Charles de Gaulle',
        'cc': 'FR',
        'city': 'Paris',
        'country': 'France',
        'distance': 7,
        'formattedAddress': ['Place Charles de Gaulle',
         '75008 Paris',
         'France'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.8737338798552,
  

In [0]:
# def Categories with Foursquare Type
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [74]:
# determine Venues 
# Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,id
0,Place Charles de Gaulle – Place de l'Étoile (P...,Plaza,48.873734,2.294984,4b291899f964a5209f9824e3
1,L’Atelier Étoile de Joël Robuchon,French Restaurant,48.872679,2.297076,4d25dd051d9fa1cda75d4309
2,Cartier,Jewelry Store,48.873480,2.297463,4bd816a009ecb7135f03487c
3,Le Chiberta,French Restaurant,48.873588,2.297808,4adcda13f964a520d23621e3
4,Pierre Hermé,Bakery,48.872494,2.296565,4c75582908dcef3bcf3a1dcf
5,Sormani,Italian Restaurant,48.875897,2.293699,4adcda11f964a520343621e3
6,Bagelstein,Bagel Shop,48.875516,2.293844,546dd0fc498e1597e843450f
7,Hôtel Splendid Étoile,Hotel,48.875023,2.293549,4bc2e2b9920eb713c0b71c2c
8,EXKi,Vegetarian / Vegan Restaurant,48.871953,2.293735,50a4f15ce4b0755d9a497c9b
9,La Terrasse,Roof Deck,48.871354,2.293712,4da05c3346194eb9b02d05b5


ratinf, tips for arc de triumphe

In [78]:

venue_id = '4adcda09f964a520de3321e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print('Rating arc de triumphe: ', result['response']['venue']['rating'])
    print('Total Tips arc de triumphe: ', result['response']['venue']['tips']['count'])
    print('Nb Likes  arc de triumphe: ', result['response']['venue']['tips']['groups'][0]['items'][0]['likes']['count'])
    print('Nb Likes / Total Tips arc de triumphe' , (result['response']['venue']['tips']['groups'][0]['items'][0]['likes']['count'])/ (result['response']['venue']['tips']['count']) *100 )
except:
    print('This venue has not been rated yet.')

Rating arc de triumphe:  9.4
Total Tips arc de triumphe:  775
Nb Likes  arc de triumphe:  102
Nb Likes / Total Tips arc de triumphe 13.161290322580646


In [80]:
venue_id = '4adcda09f964a520de3321e3'
limit = 775
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb86a2b1a4b0a00286e17d4'},
 'response': {'tips': {'count': 775,
   'items': [{'agreeCount': 0,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5ad500eb65cdf857cd873fc2',
     'createdAt': 1523908843,
     'disagreeCount': 0,
     'id': '5ad500eb65cdf857cd873fc2',
     'lang': 'ne',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'photo': {'createdAt': 1523908851,
      'height': 1920,
      'id': '5ad500f395d98617c072507a',
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'suffix': '/42448440_klthrfvffAtungLOWEbnVClcWx28NspQqGfjDJrAih0.jpg',
      'visibility': 'public',
      'width': 1440},
     'photourl': 'https://fastly.4sqi.net/img/general/original/42448440_klthrfvffAtungLOWEbnVClcWx28NspQqGfjDJrAih0.jpg',
     'text': 'Wauw! 📸🤗',
     'todo': {'count': 0},
     't

In [81]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [84]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,text,agreeCount,disagreeCount,id,user.firstName,user.id
0,Wauw! 📸🤗,0,0,5ad500eb65cdf857cd873fc2,Anthony,42448440


Spots next to arc de triumphe

In [86]:
#First, let's create the GET request URL. Name your URL url.
LIMIT = 150 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, arc_latitude, arc_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
'There are {} spots .'.format(len(results['response']['groups'][0]['items']))

'There are 69 spots .'

In [87]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b291899f964a5209f9824e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'primary': True,
    'shortName': 'Plaza'}],
  'id': '4b291899f964a5209f9824e3',
  'location': {'address': 'Place Charles de Gaulle',
   'cc': 'FR',
   'city': 'Paris',
   'country': 'France',
   'distance': 7,
   'formattedAddress': ['Place Charles de Gaulle', '75008 Paris', 'France'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 48.8737338798552,
     'lng': 2.2949838638305664}],
   'lat': 48.8737338798552,
   'lng': 2.2949838638305664,
   'neighborhood': 'Champs',
   'postalCode': '75008',
   'state': 'Île-de-France'},
  'name': "Place Charles de Gaulle – Place d

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [90]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Place Charles de Gaulle – Place de l'Étoile (Place Charles de Gaulle | Place de l'Étoile),Plaza,Place Charles de Gaulle,48.873734,2.294984,"[{'label': 'display', 'lat': 48.8737338798552, 'lng': 2.2949838638305664}]",7,75008,FR,Champs,Paris,Île-de-France,France,"[Place Charles de Gaulle, 75008 Paris, France]",NaN,4b291899f964a5209f9824e3
1,L’Atelier Étoile de Joël Robuchon,French Restaurant,133 avenue des Champs-Élysées,48.872679,2.297076,"[{'label': 'display', 'lat': 48.872678910493946, 'lng': 2.2970759868621826}]",194,75008,FR,NaN,Paris,Île-de-France,France,"[133 avenue des Champs-Élysées, 75008 Paris, France]",NaN,4d25dd051d9fa1cda75d4309
2,Cartier,Jewelry Store,154 avenue des Champs Elysées,48.873480,2.297463,"[{'label': 'display', 'lat': 48.8734797, 'lng': 2.2974633}]",181,75008,FR,NaN,Paris,Île-de-France,France,"[154 avenue des Champs Elysées, 75008 Paris, France]",NaN,4bd816a009ecb7135f03487c
3,Le Chiberta,French Restaurant,3 rue Arsène Houssaye,48.873588,2.297808,"[{'label': 'display', 'lat': 48.87358771753733, 'lng': 2.297807559370994}]",204,75008,FR,NaN,Paris,Île-de-France,France,"[3 rue Arsène Houssaye, 75008 Paris, France]",NaN,4adcda13f964a520d23621e3
4,Pierre Hermé,Bakery,Publicis Drugstore,48.872494,2.296565,"[{'label': 'display', 'lat': 48.872494479692335, 'lng': 2.296564850680936}]",183,75008,FR,NaN,Paris,Île-de-France,France,"[Publicis Drugstore (133 avenue des Champs-Élysées), 75008 Paris, France]",133 avenue des Champs-Élysées,4c75582908dcef3bcf3a1dcf
5,Sormani,Italian Restaurant,4 rue du Général Lanrezac,48.875897,2.293699,"[{'label': 'display', 'lat': 48.87589691362336, 'lng': 2.293698672623153}]",253,75017,FR,NaN,Paris,Île-de-France,France,"[4 rue du Général Lanrezac, 75017 Paris, France]",NaN,4adcda11f964a520343621e3
6,Bagelstein,Bagel Shop,8 avenue Carnot,48.875516,2.293844,"[{'label': 'display', 'lat': 48.875516, 'lng': 2.293844}]",210,75017,FR,NaN,Paris,Île-de-France,France,"[8 avenue Carnot, 75017 Paris, France]",NaN,546dd0fc498e1597e843450f
7,Hôtel Splendid Étoile,Hotel,1 avenue Carnot,48.875023,2.293549,"[{'label': 'display', 'lat': 48.875022570603235, 'lng': 2.29354917537421}]",174,75017,FR,NaN,Paris,Île-de-France,France,"[1 avenue Carnot, 75017 Paris, France]",NaN,4bc2e2b9920eb713c0b71c2c
8,EXKi,Vegetarian / Vegan Restaurant,8 avenue Kleber,48.871953,2.293735,"[{'label': 'display', 'lat': 48.871952663753056, 'lng': 2.2937352836600793}]",225,75016,FR,NaN,Paris,Île-de-France,France,"[8 avenue Kleber, 75016 Paris, France]",NaN,50a4f15ce4b0755d9a497c9b
9,La Terrasse,Roof Deck,17 avenue Kléber,48.871354,2.293712,"[{'label': 'display', 'lat': 48.871354, 'lng': 2.293712}]",287,75116,FR,NaN,Paris,Île-de-France,France,"[17 avenue Kléber (Avenue des Portugais), 75116 Paris, France]",Avenue des Portugais,4da05c3346194eb9b02d05b5


In [92]:
newTest = dataframe_filtered[['name', 'categories']]
newTest

,name,categories
0,Place Charles de Gaulle – Place de l'Étoile (Place Charles de Gaulle | Place de l'Étoile),Plaza
1,L’Atelier Étoile de Joël Robuchon,French Restaurant
2,Cartier,Jewelry Store
3,Le Chiberta,French Restaurant
4,Pierre Hermé,Bakery
5,Sormani,Italian Restaurant
6,Bagelstein,Bagel Shop
7,Hôtel Splendid Étoile,Hotel
8,EXKi,Vegetarian / Vegan Restaurant
9,La Terrasse,Roof Deck


In [93]:
print(dataframe_filtered.categories.unique())
#categories around arc de triumphe

['Plaza' 'French Restaurant' 'Jewelry Store' 'Bakery' 'Italian Restaurant'
 'Bagel Shop' 'Hotel' 'Vegetarian / Vegan Restaurant' 'Roof Deck'
 'Movie Theater' 'Cocktail Bar' 'Steakhouse' 'Electronics Store'
 'Scandinavian Restaurant' 'Tourist Information Center'
 'Molecular Gastronomy Restaurant' 'Moroccan Restaurant' 'Clothing Store'
 'Spa' 'Breton Restaurant' 'Boutique' 'Japanese Restaurant' 'Nightclub'
 'Thai Restaurant' 'Comedy Club' 'Candy Store' 'Convenience Store'
 'Asian Restaurant' 'Kids Store' 'Sandwich Place' 'Bistro' 'Café'
 'Cosmetics Shop' 'Creperie' 'Deli / Bodega']


### Map visualization

In [98]:
venues_map = folium.Map(location=[arc_latitude, arc_longitude], zoom_start=15) 
label = folium.Popup('arc de triumphe', parse_html=True)
folium.CircleMarker(
    [arc_latitude, arc_longitude],
    radius=10,
    popup=label,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

    # add the spots
for lat, lng, name, category in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name, dataframe_filtered.categories):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        fill=True,
        color= 'green',
        fill_color = 'black',
        fill_opacity=0.6
        ).add_to(venues_map)
venues_map


Conclusion : Here  are the best place around Arc de triumphe, i can continue to do it, or limit the number of spot (by the rating)

I only do it for arc de triumphe but i juste have to repeat the process for other monuments if i want a complete map with all spots of each monument.